## Mount drive to read data

In [ ]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


## Read JSON file from drive

In [ ]:
import pandas as pd
import json

with open('drive/My Drive/Capstone/data/fundamentals_results.json') as f:
    financial_records = json.load(f)

## Verify fundamental factors are the same

In [ ]:
financial_records[0]["financial_data"]["quarterly"].keys()

dict_keys(['revenue', 'cost_of_revenue', 'gross_profit', 'operating_income', 'total_assets', 'total_current_assets', 'prepaid_expenses', 'property_plant_and_equipment_net', 'retained_earnings', 'other_assets_noncurrent', 'total_liabilities', 'shareholder_equity', 'net_income', 'shares_outstanding_diluted', 'shares_outstanding_basic', 'eps_diluted', 'eps_basic', 'operating_cash_flow', 'investing_cash_flow', 'financing_cash_flow', 'research_development_expense', 'selling_general_administrative_expense', 'operating_expenses', 'non_operating_income', 'pre_tax_income', 'income_tax', 'depreciation_amortization', 'stock_based_compensation', 'dividends_paid', 'cash_on_hand', 'current_net_receivables', 'inventory', 'total_current_liabilities', 'total_non_current_liabilities', 'long_term_debt', 'goodwill', 'intangible_assets_excluding_goodwill'])

In [ ]:
financial_records[100]["financial_data"]["quarterly"].keys()

dict_keys(['revenue', 'cost_of_revenue', 'gross_profit', 'operating_income', 'total_assets', 'total_current_assets', 'prepaid_expenses', 'property_plant_and_equipment_net', 'retained_earnings', 'other_assets_noncurrent', 'total_liabilities', 'shareholder_equity', 'net_income', 'shares_outstanding_diluted', 'shares_outstanding_basic', 'eps_diluted', 'eps_basic', 'operating_cash_flow', 'investing_cash_flow', 'financing_cash_flow', 'research_development_expense', 'selling_general_administrative_expense', 'operating_expenses', 'non_operating_income', 'pre_tax_income', 'income_tax', 'depreciation_amortization', 'stock_based_compensation', 'dividends_paid', 'cash_on_hand', 'current_net_receivables', 'inventory', 'total_current_liabilities', 'total_non_current_liabilities', 'long_term_debt', 'goodwill', 'intangible_assets_excluding_goodwill'])

In [ ]:
columns_of_interest = ['operating_income', 'pre_tax_income', 'net_income']
calculated_columns = {'net_profit_margin': 'net_income/revenue', 'operating_margin': 'operating_income/revenue', 'gross_margin': 'gross_profit/revenue', 'ebitda': 'operating_income+depreciation_amortization'}
cross_table_calcualated_columns = {'cash_flow_per_share': [['free_cash_flow/shares_outstanding_basic', 'free_cash_flow/shares_outstanding_diluted'], ['free_cash_flow'],['shares_outstanding_basic', 'shares_outstanding_diluted']] }

In [ ]:
record_set = {'operating_income', 'pre_tax_income', 'net_income', 'revenue', 'gross_profit', 'shares_outstanding_basic', 'shares_outstanding_diluted', 'depreciation_amortization'}
financial_records = [record for record in financial_records if 'error' not in record]
differing_records = 0
for record in financial_records:
    pres_record_keys = set(record["financial_data"]["quarterly"].keys())
    if not pres_record_keys.issuperset(record_set):
        differing_records += 1
        print(f"{record['company_info']['ticker']} failed on data columns:", pres_record_keys.symmetric_difference(record_set))

if differing_records == 0:
    print("All record variables were similar")

All record variables were similar


## Convert JSON to Pandas dataframe

In [ ]:
from tqdm import tqdm

df = pd.DataFrame(columns=["ticker", "currency", "quarter", *record_set, 'net_profit_margin', 'operating_margin', 'gross_margin', 'ebitda'])

for record in tqdm(financial_records):
    start_quarter, end_quarter = None, None

    for variable in record_set:
        record_keys = list(record["financial_data"]["quarterly"][variable].keys())

        if len(record_keys) != 0:
            smallest_quarter_in_record = min(record_keys)
            biggest_quarter_in_record = max(record_keys)
            start_quarter = min(start_quarter, smallest_quarter_in_record) if start_quarter is not None else smallest_quarter_in_record
            end_quarter = max(end_quarter, biggest_quarter_in_record) if end_quarter is not None else biggest_quarter_in_record

    quarter_list = [start_quarter]
    start_quarter = [int(entry) for entry in start_quarter.split("Q")]
    running_quarter = start_quarter
    pres_quarter = None
    while pres_quarter != end_quarter:
        running_quarter[0] = running_quarter[0] + (running_quarter[1])//4
        running_quarter[1] = running_quarter[1]%4 + 1
        pres_quarter = f"{running_quarter[0]}Q{running_quarter[1]}"
        quarter_list.append(pres_quarter)

    for quarter in quarter_list:
        revenue = record["financial_data"]["quarterly"]["revenue"].get(quarter, None)
        gross_profit = record["financial_data"]["quarterly"]["gross_profit"].get(quarter, None)
        operating_income = record["financial_data"]["quarterly"]["operating_income"].get(quarter, None)
        net_profit = record["financial_data"]["quarterly"]["net_income"].get(quarter, None)
        operating_margin = operating_income/revenue if operating_income != None and revenue != None and revenue != 0 else None
        gross_margin = gross_profit/revenue if gross_profit != None and revenue != None and revenue != 0 else None
        net_profit_margin = net_profit/revenue if net_profit != None and revenue != None and revenue != 0 else None
        ebitda = operating_income + record["financial_data"]["quarterly"]["depreciation_amortization"].get(quarter, 0) if operating_income != None else None
        df.loc[len(df.index)] = [record['company_info']['ticker'], record['currency'], quarter, *[record["financial_data"]["quarterly"][variable].get(quarter, None) for variable in record_set], net_profit_margin, operating_margin, gross_margin, ebitda]

df

100%|██████████| 491/491 [04:07<00:00,  1.99it/s]


,ticker,currency,quarter,pre_tax_income,operating_income,revenue,shares_outstanding_basic,shares_outstanding_diluted,depreciation_amortization,net_income,gross_profit,net_profit_margin,operating_margin,gross_margin,ebitda
0,MMM,USD,2008Q2,1416000000,1449000000,6739000000,702100000,712000000,None,945000000,3229000000,0.140229,0.215017,0.479151,1449000000
1,MMM,USD,2008Q3,1489000000,1513000000,6558000000,695500000,703100000,None,991000000,3126000000,0.151113,0.230711,0.47667,1513000000
2,MMM,USD,2008Q4,None,None,None,None,None,None,None,None,NaN,NaN,NaN,None
3,MMM,USD,2009Q1,759000000,803000000,5089000000,693500000,695900000,271000000,518000000,2317000000,0.101788,0.157791,0.455296,1074000000
4,MMM,USD,2009Q2,1143000000,1191000000,5719000000,696800000,700300000,None,783000000,2742000000,0.136912,0.208253,0.479454,1191000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28042,ZTS,USD,2022Q3,667000000,667000000,2002000000,467800000,469100000,115000000,529000000,1395000000,0.264236,0.333167,0.696803,782000000
28043,ZTS,USD,2022Q4,592000000,592000000,2040000000,468891000,470385000,119000000,461000000,1387000000,0.22598,0.290196,0.679902,711000000
28044,ZTS,USD,2023Q1,697000000,697000000,2000000000,463500000,464600000,120000000,552000000,1412000000,0.276,0.3485,0.706,817000000
28045,ZTS,USD,2023Q2,872000000,872000000,2180000000,461900000,462900000,121000000,671000000,1573000000,0.307798,0.4,0.72156,993000000


## Filter redundant columns

In [ ]:
df["currency"].unique()

array(['USD'], dtype=object)

In [ ]:
df = df.drop(columns = 'currency')

## Group fundamentals by company ticker

In [ ]:
grouped_df = df.groupby('ticker')

grouped_df.get_group('A')

,ticker,quarter,pre_tax_income,operating_income,revenue,shares_outstanding_basic,shares_outstanding_diluted,depreciation_amortization,net_income,gross_profit,net_profit_margin,operating_margin,gross_margin,ebitda
536,A,2008Q2,215000000,218000000,1444000000,363052334000000000000,373081404000000000000,None,169000000,803000000,0.117036,0.15097,0.556094,218000000
537,A,2008Q3,None,None,None,None,None,None,None,None,None,None,None,None
538,A,2008Q4,27000000,24000000,1166000000,352020357000000,353023264000000,41000000,64000000,589000000,0.054889,0.020583,0.505146,65000000
539,A,2009Q1,-58000000,-47000000,1091000000,345000008000000,345000008000000,None,-101000000,530000000,-0.092576,-0.04308,0.485793,-47000000
540,A,2009Q2,-41000000,-1000000,1057000000,345000000000000,345000000000000,None,-19000000,539000000,-0.017975,-0.000946,0.509934,-1000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
593,A,2022Q3,455000000,471000000,1849000000,299000000,300000000,None,368000000,1012000000,0.199027,0.254732,0.547323,471000000
594,A,2022Q4,410000000,426000000,1756000000,296000000,297000000,67000000,352000000,968000000,0.200456,0.242597,0.551253,493000000
595,A,2023Q1,377000000,383000000,1717000000,296000000,297000000,None,302000000,924000000,0.175888,0.223063,0.538148,383000000
596,A,2023Q2,132000000,133000000,1672000000,294000000,295000000,None,111000000,658000000,0.066388,0.079545,0.393541,133000000


## Merge in GICS Industry, Sub-industry, Sector, and Industry Group classifications for each ticker

In [ ]:
s_and_p_df = pd.read_csv('drive/My Drive/Capstone/data/s&pconstituents.csv')

s_and_p_df

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
3,ABBV,AbbVie,Health Care,Biotechnology,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989
...,...,...,...,...,...,...,...,...
498,YUM,Yum! Brands,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,1041061,1997
499,ZBRA,Zebra Technologies,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",2019-12-23,877212,1969
500,ZBH,Zimmer Biomet,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,1136869,1927
501,ZION,Zions Bancorporation,Financials,Regional Banks,"Salt Lake City, Utah",2001-06-22,109380,1873


In [ ]:
gics_classifications_df = pd.read_csv('drive/My Drive/Capstone/data/gics_classifications.csv')

gics_classifications_df

,Sector,Industry Group,Industry,Sub-Industry
0,Energy,Energy,Energy Equipment & Services,Oil & Gas Drilling
1,Energy,Energy,Energy Equipment & Services,Oil & Gas Equipment & Services
2,Energy,Energy,"Oil, Gas & Consumable Fuels",Integrated Oil & Gas
3,Energy,Energy,"Oil, Gas & Consumable Fuels",Oil & Gas Exploration & Production
4,Energy,Energy,"Oil, Gas & Consumable Fuels",Oil & Gas Refining & Marketing
...,...,...,...,...
159,Real Estate,Equity Real Estate Investment Trusts (REITs),Specialized REITs,Data Center REITs
160,Real Estate,Real Estate Management & Development,Real Estate Management & Development,Diversified Real Estate Activities
161,Real Estate,Real Estate Management & Development,Real Estate Management & Development,Real Estate Operating Companies
162,Real Estate,Real Estate Management & Development,Real Estate Management & Development,Real Estate Development


## Merge other levels of classification to s&p 500 list

In [ ]:
s_and_p_with_gics_df = s_and_p_df.merge(gics_classifications_df, left_on='GICS Sub-Industry', right_on='Sub-Industry').drop(columns=['GICS Sector', 'GICS Sub-Industry'])

s_and_p_with_gics_df

,Symbol,Security,Headquarters Location,Date added,CIK,Founded,Sector,Industry Group,Industry,Sub-Industry
0,MMM,3M,"Saint Paul, Minnesota",1957-03-04,66740,1902,Industrials,Capital Goods,Industrial Conglomerates,Industrial Conglomerates
1,GE,General Electric,"Boston, Massachusetts",1957-03-04,40545,1892,Industrials,Capital Goods,Industrial Conglomerates,Industrial Conglomerates
2,HON,Honeywell,"Charlotte, North Carolina",1957-03-04,773840,1906,Industrials,Capital Goods,Industrial Conglomerates,Industrial Conglomerates
3,AOS,A. O. Smith,"Milwaukee, Wisconsin",2017-07-26,91142,1916,Industrials,Capital Goods,Building Products,Building Products
4,ALLE,Allegion,"Dublin, Ireland",2013-12-02,1579241,1908,Industrials,Capital Goods,Building Products,Building Products
...,...,...,...,...,...,...,...,...,...,...
498,TMUS,T-Mobile US,"Bellevue, Washington",2019-07-15,1283699,1994,Communication Services,Telecommunication Services,Wireless Telecommunication Services,Wireless Telecommunication Services
499,UBER,Uber,"San Francisco, California",2023-12-18,1543151,2009,Industrials,Transportation,Ground Transportation,Passenger Ground Transportation
500,WBA,Walgreens Boots Alliance,"Deerfield, Illinois",1979-12-31,1618921,2014,Consumer Staples,Consumer Staples Distribution & Retail,Consumer Staples Distribution & Retail,Drug Retail
501,WY,Weyerhaeuser,"Seattle, Washington",1979-10-01,106535,1900,Real Estate,Equity Real Estate Investment Trusts (REITs),Specialized REITs,Timber REITs


## Add industry classifications to fundamentals data frame

In [ ]:
df = df.merge(s_and_p_with_gics_df, left_on = 'ticker', right_on = 'Symbol').drop(columns=['Symbol', 'Security', 'Headquarters Location', 'CIK', 'Founded', 'Date added'])

df

,ticker,quarter,pre_tax_income,operating_income,revenue,shares_outstanding_basic,shares_outstanding_diluted,depreciation_amortization,net_income,gross_profit,net_profit_margin,operating_margin,gross_margin,ebitda,Sector,Industry Group,Industry,Sub-Industry
0,MMM,2008Q2,1416000000,1449000000,6739000000,702100000,712000000,None,945000000,3229000000,0.140229,0.215017,0.479151,1449000000,Industrials,Capital Goods,Industrial Conglomerates,Industrial Conglomerates
1,MMM,2008Q3,1489000000,1513000000,6558000000,695500000,703100000,None,991000000,3126000000,0.151113,0.230711,0.47667,1513000000,Industrials,Capital Goods,Industrial Conglomerates,Industrial Conglomerates
2,MMM,2008Q4,None,None,None,None,None,None,None,None,NaN,NaN,NaN,None,Industrials,Capital Goods,Industrial Conglomerates,Industrial Conglomerates
3,MMM,2009Q1,759000000,803000000,5089000000,693500000,695900000,271000000,518000000,2317000000,0.101788,0.157791,0.455296,1074000000,Industrials,Capital Goods,Industrial Conglomerates,Industrial Conglomerates
4,MMM,2009Q2,1143000000,1191000000,5719000000,696800000,700300000,None,783000000,2742000000,0.136912,0.208253,0.479454,1191000000,Industrials,Capital Goods,Industrial Conglomerates,Industrial Conglomerates
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28042,ZTS,2022Q3,667000000,667000000,2002000000,467800000,469100000,115000000,529000000,1395000000,0.264236,0.333167,0.696803,782000000,Health Care,"Pharmaceuticals, Biotechnology & Life Sciences",Pharmaceuticals,Pharmaceuticals
28043,ZTS,2022Q4,592000000,592000000,2040000000,468891000,470385000,119000000,461000000,1387000000,0.22598,0.290196,0.679902,711000000,Health Care,"Pharmaceuticals, Biotechnology & Life Sciences",Pharmaceuticals,Pharmaceuticals
28044,ZTS,2023Q1,697000000,697000000,2000000000,463500000,464600000,120000000,552000000,1412000000,0.276,0.3485,0.706,817000000,Health Care,"Pharmaceuticals, Biotechnology & Life Sciences",Pharmaceuticals,Pharmaceuticals
28045,ZTS,2023Q2,872000000,872000000,2180000000,461900000,462900000,121000000,671000000,1573000000,0.307798,0.4,0.72156,993000000,Health Care,"Pharmaceuticals, Biotechnology & Life Sciences",Pharmaceuticals,Pharmaceuticals


## Group fundamentals by company ticker

In [ ]:
grouped_df = df.groupby('ticker')

grouped_df.get_group('A')

,ticker,quarter,pre_tax_income,operating_income,revenue,shares_outstanding_basic,shares_outstanding_diluted,depreciation_amortization,net_income,gross_profit,net_profit_margin,operating_margin,gross_margin,ebitda,Sector,Industry Group,Industry,Sub-Industry
536,A,2008Q2,215000000,218000000,1444000000,363052334000000000000,373081404000000000000,None,169000000,803000000,0.117036,0.15097,0.556094,218000000,Health Care,"Pharmaceuticals, Biotechnology & Life Sciences",Life Sciences Tools & Services,Life Sciences Tools & Services
537,A,2008Q3,None,None,None,None,None,None,None,None,None,None,None,None,Health Care,"Pharmaceuticals, Biotechnology & Life Sciences",Life Sciences Tools & Services,Life Sciences Tools & Services
538,A,2008Q4,27000000,24000000,1166000000,352020357000000,353023264000000,41000000,64000000,589000000,0.054889,0.020583,0.505146,65000000,Health Care,"Pharmaceuticals, Biotechnology & Life Sciences",Life Sciences Tools & Services,Life Sciences Tools & Services
539,A,2009Q1,-58000000,-47000000,1091000000,345000008000000,345000008000000,None,-101000000,530000000,-0.092576,-0.04308,0.485793,-47000000,Health Care,"Pharmaceuticals, Biotechnology & Life Sciences",Life Sciences Tools & Services,Life Sciences Tools & Services
540,A,2009Q2,-41000000,-1000000,1057000000,345000000000000,345000000000000,None,-19000000,539000000,-0.017975,-0.000946,0.509934,-1000000,Health Care,"Pharmaceuticals, Biotechnology & Life Sciences",Life Sciences Tools & Services,Life Sciences Tools & Services
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
593,A,2022Q3,455000000,471000000,1849000000,299000000,300000000,None,368000000,1012000000,0.199027,0.254732,0.547323,471000000,Health Care,"Pharmaceuticals, Biotechnology & Life Sciences",Life Sciences Tools & Services,Life Sciences Tools & Services
594,A,2022Q4,410000000,426000000,1756000000,296000000,297000000,67000000,352000000,968000000,0.200456,0.242597,0.551253,493000000,Health Care,"Pharmaceuticals, Biotechnology & Life Sciences",Life Sciences Tools & Services,Life Sciences Tools & Services
595,A,2023Q1,377000000,383000000,1717000000,296000000,297000000,None,302000000,924000000,0.175888,0.223063,0.538148,383000000,Health Care,"Pharmaceuticals, Biotechnology & Life Sciences",Life Sciences Tools & Services,Life Sciences Tools & Services
596,A,2023Q2,132000000,133000000,1672000000,294000000,295000000,None,111000000,658000000,0.066388,0.079545,0.393541,133000000,Health Care,"Pharmaceuticals, Biotechnology & Life Sciences",Life Sciences Tools & Services,Life Sciences Tools & Services


## Save parsed dataframe as CSV

In [ ]:
df.to_csv("parsed_fundamentals.csv", index=False)
!cp "parsed_fundamentals.csv" "drive/My Drive/Capstone/data/"

## Scrape fundamentals data

In [ ]:
variables = {'Revenue': ['revenue'],
 'Gross Profit': ['net_income+expenses_total'],
 'Net Income, GAAP': ['net_income'],
 'Gross Margin': ['(net_income+expenses_total)/revenue'],
 'Net Profit Margin': ['q_profit_margin'],
 'Book Value per share': ['book_value_per_share'],
 'Dividend per share': ['dividend'],
 'Current Ratio': ['debt_to_equity', 'debt_to_capital'],
 'Cash Flow from Operations': ['cash_from_operations'],
 'Cash Flow from Investing Activities': ['cash_from_investing'],
 'Cash Flow from Financing Activities': ['cash_from_financing'],
 'Net Debt': ['liabilities_total+debt_lt_total_ttm-cash']}

supported_columns_of_interest = ["revenue", "net_income", "expenses_total", "q_profit_margin", "book_value_per_share", "dividend", "debt_to_equity", "cash_from_operations", "cash_from_investing", "cash_from_financing", "free_cash_flow", "share_holders_equity", "cash"]
composite_columns = {"gross_profit": "net_income+expenses_total", "gross_margin": "gross_profit/revenue", "net_debt": "debt_to_equity*share_holders_equity - cash"}

In [ ]:
import requests
import json

requested_fundamentals_df = pd.DataFrame()
for symbol in tqdm(s_and_p_df["Symbol"]):
    symbol_df = pd.DataFrame({'symbol' : []})
    for col in supported_columns_of_interest:
        params = {
            'ticker': symbol,
            'wrapper': col,
            'addl_settings': 'time_period=2004,2021,2022,2023',
        }

        response = requests.get('https://www.zacks.com//data_handler/charts/', params=params)
        try:
            value_json = json.loads(response.text)[col]
            new_df = pd.DataFrame.from_dict(dict(sorted([(k, v) for k,v in value_json.items()])), orient='index',
                          columns=[col])
            symbol_df = pd.concat([symbol_df, new_df], axis=1)
        except:
            print(f"Failed for symbol{symbol} on column {col}")
    symbol_df = symbol_df.reset_index().rename(columns={"index":"fiscal_date"})
    symbol_df["symbol"] = symbol

    requested_fundamentals_df = pd.concat([requested_fundamentals_df, symbol_df], sort=False)

requested_fundamentals_df

In [ ]:
import pandas as pd
import numpy as np

# Convert to datetiem for conversion to quarterly later
requested_fundamentals_df["fiscal_date"] = pd.to_datetime(requested_fundamentals_df["fiscal_date"]).dt.normalize()

# Replace the N/A with NaN for each checks
requested_fundamentals_df = requested_fundamentals_df.sort_values(['symbol', 'fiscal_date'], ignore_index=True).replace('N/A', np.nan)

# Fill the quarterly results with the earlier dividend
requested_fundamentals_df['dividend'] = (requested_fundamentals_df.groupby('symbol', group_keys=False)['dividend'].apply(lambda s: s.ffill()))

# Create composite columns
requested_fundamentals_df['gross_profit'] = requested_fundamentals_df['net_income'].astype(float) + requested_fundamentals_df['expenses_total'].astype(float)
requested_fundamentals_df['gross_margin'] = requested_fundamentals_df['gross_profit'].astype(float)/requested_fundamentals_df['revenue'].astype(float)
requested_fundamentals_df['net_debt'] = requested_fundamentals_df['debt_to_equity'].astype(float)*requested_fundamentals_df['share_holders_equity'].astype(float) - requested_fundamentals_df['cash'].astype(float)

# keep only columns with values in more than 3 places so the daily dividend effect is removed: more than (date, symbol, dividend)
requested_fundamentals_df = requested_fundamentals_df.dropna(thresh=4)
requested_fundamentals_df

In [ ]:
grouped_requested_fundamentals_df = requested_fundamentals_df.groupby('symbol')

grouped_requested_fundamentals_df.get_group('A')

In [ ]:
# Write to CSV
requested_fundamentals_df.to_csv("zacks_fundamentals.csv", index=False)
!cp "zacks_fundamentals.csv" "drive/My Drive/Capstone/data/"

## Scrape estimates for API

In [ ]:
!pip -q install selenium
!apt-get -qq update
!apt -qq install chromium-chromedrive

In [ ]:
id_symbol = {}
for symbol in tqdm(s_and_p_df["Symbol"]):
    response = requests.get(f"https://seekingalpha.com/api/v3/historical_prices?filter[ticker][slug]={symbol}&filter[for_date]=2023-12-07&sort=as_of_date")
    id = json.loads(response.text)["included"][0]["id"]
    id_symbol[id] = symbol

id_symbol

In [ ]:
import pandas as pd
import numpy as np
import math
import datetime
import time
import requests
import json
from selenium.webdriver import Firefox
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
import os
from selenium.webdriver.common.by import By
import urllib
#import geckodriver_autoinstaller

class Zacks():
    '''
    This class scrapes Zacks.com to get earnings data from a companies earnings reports.
    '''
    def __init__(self,driver):
        #geckodriver_autoinstaller.install()
        self.driver = driver

    def get_ids(self, id_to_symbol):

        # Start firefox browser in selenium
        #browser = self.__get_browser()
        browser = self.driver
        id_to_symbol = {}
        for symbol in df:
            browser.get(f'https://seekingalpha.com/api/v3/historical_prices?filter[ticker][slug]={symbol.lower()}&filter[for_date]=2023-12-07&sort=as_of_date')
            soup = BeautifulSoup(browser.page_source, 'html.parser')
            try:
                id = json.loads(soup.find("body").text)["included"][0]["id"]
            except:
                print(symbol)
                continue
            id_to_symbol[id] = symbol

        estimates_range = ",".join(str(x) for x in range(-40,1))

        id_keys = list(id_to_symbol.keys())
        ticker_estimates = {}
        for i in range(0, len(id_keys), 20):
            ticker_ids = ",".join(id_keys[i:i+20])

            params = {
            'estimates_data_items': 'eps_normalized_actual,eps_normalized_consensus_low,eps_normalized_consensus_mean,eps_normalized_consensus_high,eps_normalized_num_of_estimates,revenue_consensus_mean,ffo_consensus_mean',
            'period_type': 'quarterly',
            'relative_periods': estimates_range,
            'ticker_ids': ticker_ids,
            }
            url_params = urllib.parse.urlencode(params)
            url = 'https://seekingalpha.com/api/v3/symbol_data/estimates?'+url_params
            browser.get(url)
            soup = BeautifulSoup(browser.page_source, 'html.parser')
            try:
                estimates = json.loads(soup.find("body").text)["estimates"]
            except:
                print(url)
                continue
            ticker_estimates.update(estimates)
        # Close browser and process results for output
        browser.close()

        return ticker_estimates


In [ ]:
from selenium import webdriver

chrome_options = webdriver.ChromeOptions()

chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--disable-blink-features=AutomationControlled')

driver = webdriver.Chrome(options=chrome_options)
scraper = Zacks(driver)
eps_estimates = scraper.get_ids(id_symbol)

In [ ]:
df_columns = ['eps_normalized_actual',
              'eps_normalized_consensus_low',
              'eps_normalized_consensus_mean',
              'eps_normalized_consensus_high',
              'eps_normalized_num_of_estimates',
              'revenue_consensus_mean',
              'ffo_consensus_mean']
estimates_df = pd.DataFrame()
for symbol in id_symbol.keys():
    symbol_df = pd.DataFrame({'symbol' : []})
    for col in df_columns:
        if col in eps_estimates[symbol].keys():
            new_df = pd.DataFrame.from_dict(dict(sorted([(str(v[0]["period"]["calendaryear"])+"Q"+str(v[0]["period"]["calendarquarter"]), float(v[0]['dataitemvalue'])) for v in eps_estimates[symbol][col].values()])), orient='index',
                              columns=[col])
            symbol_df = pd.concat([symbol_df, new_df], axis=1)

    calendar_df = pd.DataFrame.from_dict(dict(sorted([(str(v[0]["period"]["calendaryear"])+"Q"+str(v[0]["period"]["calendarquarter"]), v[0]["period"]["periodenddate"]) for v in eps_estimates[symbol]['eps_normalized_actual'].values()])), orient='index',
                              columns=['fiscal_end_date'])
    symbol_df = symbol_df.join(calendar_df)
    symbol_df = symbol_df.reset_index().rename(columns={"index":"fiscal_year_and_quarter"})
    symbol_df["symbol"] = id_symbol[symbol]

    estimates_df = pd.concat([estimates_df, symbol_df], sort=False, ignore_index=True)

# Convert to datetime for use in conversion to quarterly later
estimates_df["fiscal_end_date"] = pd.to_datetime(estimates_df["fiscal_end_date"], utc=True).dt.normalize()

estimates_df

In [ ]:
# Write to CSV
estimates_df.to_csv("ticker_estimates.csv", index=False)
!cp "ticker_estimates.csv" "drive/My Drive/Capstone/data/"

In [ ]:
estimates_df = pd.read_csv("drive/My Drive/Capstone/data/ticker_estimates.csv")
requested_fundamentals_df = pd.read_csv("drive/My Drive/Capstone/data/zacks_fundamentals.csv")

In [ ]:
requested_fundamentals_df["fiscal_date"] = pd.to_datetime(requested_fundamentals_df["fiscal_date"], utc=True).dt.normalize()

In [ ]:
estimates_df["fiscal_end_date"] = pd.to_datetime(estimates_df["fiscal_end_date"], utc=True).dt.normalize()

## Merge return with estimates

In [ ]:
sp500_prices = pd.read_csv("HistoricalPrices.csv")[["Date", " Close"]]
sp500_prices = sp500_prices.iloc[::-1].reset_index(drop=True)
sp500_prices['Date'] = pd.to_datetime(sp500_prices['Date'], utc = True)
sp500_prices = sp500_prices.set_index('Date').asfreq('D', method='ffill').reset_index()
sp500_prices

,Date,Close
0,2012-09-07 00:00:00+00:00,1437.92
1,2012-09-08 00:00:00+00:00,1437.92
2,2012-09-09 00:00:00+00:00,1437.92
3,2012-09-10 00:00:00+00:00,1429.08
4,2012-09-11 00:00:00+00:00,1433.56
...,...,...
4103,2023-12-02 00:00:00+00:00,4594.63
4104,2023-12-03 00:00:00+00:00,4594.63
4105,2023-12-04 00:00:00+00:00,4569.78
4106,2023-12-05 00:00:00+00:00,4567.18


In [ ]:
sp500_prices['Date'] = sp500_prices['Date'].dt.normalize()
sp500_prices['prev_60_day_return'] = None

for idx, row in sp500_prices.iterrows():
    if idx < 60:
        continue
    sixty_days_back = sp500_prices.loc[idx-60,' Close']
    sp500_prices.loc[idx,'prev_60_day_return'] = (sp500_prices.loc[idx-1,' Close'] - sixty_days_back)/sixty_days_back

sp500_prices

,Date,Close,prev_60_day_return
0,2012-09-07 00:00:00+00:00,1437.92,None
1,2012-09-08 00:00:00+00:00,1437.92,None
2,2012-09-09 00:00:00+00:00,1437.92,None
3,2012-09-10 00:00:00+00:00,1429.08,None
4,2012-09-11 00:00:00+00:00,1433.56,None
...,...,...,...
4103,2023-12-02 00:00:00+00:00,4594.63,0.086342
4104,2023-12-03 00:00:00+00:00,4594.63,0.077603
4105,2023-12-04 00:00:00+00:00,4569.78,0.07901
4106,2023-12-05 00:00:00+00:00,4567.18,0.060643


In [ ]:
sp500_prices.dtypes

Date                  datetime64[ns, UTC]
 Close                            float64
prev_60_day_return                 object
dtype: object

In [ ]:
returns_with_estimates = estimates_df.merge(sp500_prices, how='left', left_on=['fiscal_end_date'], right_on = ['Date']).drop(columns = ['Date'])
returns_with_estimates

,fiscal_year_and_quarter,symbol,eps_normalized_actual,eps_normalized_consensus_low,eps_normalized_consensus_mean,eps_normalized_consensus_high,eps_normalized_num_of_estimates,revenue_consensus_mean,fiscal_end_date,ffo_consensus_mean,Close,prev_60_day_return
0,2013Q3,MMM,1.78,1.7000,1.75357,1.79667,16.0,7.854874e+09,2013-09-30 00:00:00+00:00,NaN,1681.55,-0.008858
1,2013Q4,MMM,1.62,1.5900,1.61549,1.65000,16.0,7.713274e+09,2013-12-31 00:00:00+00:00,NaN,1848.36,0.045089
2,2014Q1,MMM,1.79,1.7500,1.79508,1.84000,13.0,7.970312e+09,2014-03-31 00:00:00+00:00,NaN,1872.34,0.035353
3,2014Q2,MMM,1.91,1.8811,1.90781,1.92000,13.0,8.087488e+09,2014-06-30 00:00:00+00:00,NaN,1960.23,0.041026
4,2014Q3,MMM,1.98,1.9200,1.96147,2.00000,12.0,8.223700e+09,2014-09-30 00:00:00+00:00,NaN,1972.29,0.027349
...,...,...,...,...,...,...,...,...,...,...,...,...
20068,2022Q3,ZTS,1.21,1.1600,1.23581,1.31000,13.0,2.081859e+09,2022-09-30 00:00:00+00:00,NaN,3585.62,-0.116097
20069,2022Q4,ZTS,1.15,1.1000,1.14704,1.18000,14.0,2.005602e+09,2022-12-31 00:00:00+00:00,NaN,3839.50,-0.004305
20070,2023Q1,ZTS,1.31,1.1600,1.25324,1.31000,13.0,2.015332e+09,2023-03-31 00:00:00+00:00,NaN,4109.31,0.008228
20071,2023Q2,ZTS,1.41,1.2000,1.30545,1.36000,13.0,2.157296e+09,2023-06-30 00:00:00+00:00,NaN,4450.38,0.054841


## Merge fundamentals, returns, and estimates

In [ ]:
merged_dataframe = df.merge(returns_with_estimates, how = 'outer', left_on=['ticker','quarter'], right_on = ['symbol','fiscal_year_and_quarter']).merge(requested_fundamentals_df, left_on=['symbol','fiscal_end_date'], right_on = ['symbol','fiscal_date'])
merged_dataframe['symbol'] = merged_dataframe['symbol'].fillna(merged_dataframe['ticker'])
merged_dataframe['quarter'] = merged_dataframe['quarter'].fillna(merged_dataframe['fiscal_year_and_quarter'])
merged_dataframe['Sector'] = merged_dataframe.groupby('symbol')['Sector'].fillna(method='ffill')
merged_dataframe['Industry Group'] = merged_dataframe.groupby('symbol')['Industry Group'].fillna(method='ffill')
merged_dataframe['Industry'] = merged_dataframe.groupby('symbol')['Industry'].fillna(method='ffill')
merged_dataframe['Sub-Industry'] = merged_dataframe.groupby('symbol')['Sub-Industry'].fillna(method='ffill')
merged_dataframe

,ticker,quarter,gross_profit_x,shares_outstanding_diluted,net_income_x,pre_tax_income,operating_income,shares_outstanding_basic,depreciation_amortization,revenue_x,...,debt_to_equity,cash_from_operations,cash_from_investing,cash_from_financing,free_cash_flow,share_holders_equity,cash,gross_profit_y,gross_margin_y,net_debt
0,MMM,2013Q3,3768000000,691800000,1230000000,1716000000,1739000000,679800000,None,7916000000,...,19.357,1151.0,38.0,-1916.0,815.0,18252.0,3311.0,7911.0,0.999368,349992.964
1,MMM,2013Q4,3593000000,693600000,1103000000,1558000000,1579000000,681900000,None,7569000000,...,24.103,1993.0,-154.0,-1603.0,1492.0,17948.0,3337.0,7567.0,0.999736,429263.644
2,MMM,2014Q1,3800000000,674500000,1207000000,1688000000,1716000000,661500000,350000000,7831000000,...,24.554,1092.0,-287.0,-1399.0,802.0,17924.0,2814.0,7822.0,0.998851,437291.896
3,MMM,2014Q2,3950000000,664600000,1267000000,1820000000,1856000000,652000000,None,8134000000,...,29.827,1640.0,-292.0,-1185.0,1334.0,17846.0,3009.0,8127.0,0.999139,529283.642
4,MMM,2014Q3,3932000000,657900000,1303000000,1880000000,1901000000,645300000,None,8137000000,...,32.241,1711.0,1.0,-1889.0,1420.0,16206.0,2696.0,8136.0,0.999877,519801.646
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19963,NaN,2023Q3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-136.170,477.0,-30.0,-223.0,420.0,-8190.0,656.0,1726.0,1.010539,1114576.300
19964,NaN,2023Q3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,70.395,-35.0,-14.0,43.0,-49.0,3013.0,61.0,960.0,1.004184,212039.135
19965,NaN,2023Q3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,40.960,337.6,-165.6,-196.0,188.8,12518.0,292.1,1757.2,1.002053,512445.180
19966,NaN,2023Q3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,100.230,311.0,-148.0,-164.0,280.0,5315.0,3862.0,1190.0,1.000000,528860.450


In [ ]:
merged_dataframe.groupby('quarter').count()[["eps_normalized_actual", "eps_normalized_consensus_mean"]]

,eps_normalized_actual,eps_normalized_consensus_mean
quarter,,
2010Q4,1,1
2011Q1,1,1
2011Q2,1,1
2011Q3,1,1
2011Q4,1,1
2012Q1,1,1
2012Q2,1,1
2012Q3,1,1
2012Q4,1,1


In [ ]:
merged_dataframe["cash_flow_per_share"] = merged_dataframe["free_cash_flow"].astype(float)/merged_dataframe["shares_outstanding_diluted"].astype(float)
merged_dataframe = merged_dataframe.drop(columns=['ticker', 'cash', 'expenses_total', 'fiscal_date', 'fiscal_end_date', 'fiscal_year_and_quarter', 'depreciation_amortization', 'shares_outstanding_diluted', 'shares_outstanding_basic', 'share_holders_equity', ' Close'])
merged_dataframe

,quarter,gross_profit_x,net_income_x,pre_tax_income,operating_income,revenue_x,net_profit_margin,operating_margin,gross_margin_x,ebitda,...,dividend,debt_to_equity,cash_from_operations,cash_from_investing,cash_from_financing,free_cash_flow,gross_profit_y,gross_margin_y,net_debt,cash_flow_per_share
0,2013Q3,3768000000,1230000000,1716000000,1739000000,7916000000,0.155382,0.219682,0.475998,1739000000,...,NaN,19.357,1151.0,38.0,-1916.0,815.0,7911.0,0.999368,349992.964,0.000001
1,2013Q4,3593000000,1103000000,1558000000,1579000000,7569000000,0.145726,0.208614,0.474699,1579000000,...,2.54,24.103,1993.0,-154.0,-1603.0,1492.0,7567.0,0.999736,429263.644,0.000002
2,2014Q1,3800000000,1207000000,1688000000,1716000000,7831000000,0.154131,0.219129,0.485251,2066000000,...,3.42,24.554,1092.0,-287.0,-1399.0,802.0,7822.0,0.998851,437291.896,0.000001
3,2014Q2,3950000000,1267000000,1820000000,1856000000,8134000000,0.155766,0.228178,0.485616,1856000000,...,3.42,29.827,1640.0,-292.0,-1185.0,1334.0,8127.0,0.999139,529283.642,0.000002
4,2014Q3,3932000000,1303000000,1880000000,1901000000,8137000000,0.160133,0.233624,0.483225,1901000000,...,3.42,32.241,1711.0,1.0,-1889.0,1420.0,8136.0,0.999877,519801.646,0.000002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19963,2023Q3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.42,-136.170,477.0,-30.0,-223.0,420.0,1726.0,1.010539,1114576.300,NaN
19964,2023Q3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.00,70.395,-35.0,-14.0,43.0,-49.0,960.0,1.004184,212039.135,NaN
19965,2023Q3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.96,40.960,337.6,-165.6,-196.0,188.8,1757.2,1.002053,512445.180,NaN
19966,2023Q3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.64,100.230,311.0,-148.0,-164.0,280.0,1190.0,1.000000,528860.450,NaN


In [ ]:
# get all columns except the one with the date type
cols = merged_dataframe.columns.difference(['Sector', 'Industry Group', 'Industry', 'Sub-Industry', 'quarter', 'symbol'])
# convert the relevant columns
merged_dataframe[cols] = merged_dataframe[cols].astype(float)
merged_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19968 entries, 0 to 19967
Data columns (total 37 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   quarter                          19968 non-null  object 
 1   gross_profit_x                   16148 non-null  float64
 2   net_income_x                     18285 non-null  float64
 3   pre_tax_income                   17088 non-null  float64
 4   operating_income                 17861 non-null  float64
 5   revenue_x                        17705 non-null  float64
 6   net_profit_margin                17439 non-null  float64
 7   operating_margin                 17137 non-null  float64
 8   gross_margin_x                   16030 non-null  float64
 9   ebitda                           17861 non-null  float64
 10  Sector                           19460 non-null  object 
 11  Industry Group                   19460 non-null  object 
 12  Industry          

In [ ]:
# Write to CSV
merged_dataframe.to_csv("final_with_return_estimates_fundamentals_classes.csv", index=False)
!cp "final_with_return_estimates_fundamentals_classes.csv" "drive/My Drive/Capstone/data/"